# 📈 Crypto Trend Classifier with RSI & Fibonacci Zones

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Load dataset
df = pd.read_csv(r'R:\GPT\AI-Pivot\Jupiter\btc.csv')

# Clean columns and rename if needed
df.columns = df.columns.str.strip()
df.rename(columns={'Timestamp': 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

In [2]:
# Feature: daily return
df['Return'] = df['Close'].pct_change()

# Feature: RSI (Relative Strength Index)
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = compute_rsi(df['Close'])

In [3]:
# Feature: Fibonacci Retracement Zones
window = 20
df['Fib_Max'] = df['Close'].rolling(window=window).max()
df['Fib_Min'] = df['Close'].rolling(window=window).min()
df['Fib_38'] = df['Fib_Max'] - 0.382 * (df['Fib_Max'] - df['Fib_Min'])
df['Fib_50'] = df['Fib_Max'] - 0.500 * (df['Fib_Max'] - df['Fib_Min'])
df['Fib_62'] = df['Fib_Max'] - 0.618 * (df['Fib_Max'] - df['Fib_Min'])

# Feature: is price in golden retracement zone?
df['In_Fib_Zone'] = ((df['Close'] >= df['Fib_62']) & (df['Close'] <= df['Fib_38'])).astype(int)

In [4]:
# Target: will price go up tomorrow?
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df = df.dropna()  # Drop NaNs from rolling calculations and shift

# Features and labels
X = df[['Return', 'RSI', 'In_Fib_Zone']]
y = df['Target']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6055104997421356
Classification Report:
               precision    recall  f1-score   support

           0       0.61      1.00      0.75    799553
           1       0.00      0.00      0.00    520908

    accuracy                           0.61   1320461
   macro avg       0.30      0.50      0.38   1320461
weighted avg       0.37      0.61      0.46   1320461



C:\Users\papad\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\papad\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\papad\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capital

## ✅ Summary
This notebook:
- Calculates RSI and Fibonacci retracement zones
- Uses those as features along with returns
- Predicts whether price will go up the next day

You can now visualize signals or switch to more advanced models (e.g., XGBoost, LSTM)